In [1]:
import cv2

In [10]:
vid = cv2.VideoCapture('Dino.avi')

In [15]:
vid.read?

Docstring:
read([, image]) -> retval, image
.   @brief Grabs, decodes and returns the next video frame.
.   
.       @param [out] image the video frame is returned here. If no frames has been grabbed the image will be empty.
.       @return `false` if no frames has been grabbed
.   
.       The method/function combines VideoCapture::grab() and VideoCapture::retrieve() in one call. This is the
.       most convenient method for reading video files or capturing data from decode and returns the just
.       grabbed frame. If no frames has been grabbed (camera has been disconnected, or there are no more
.       frames in video file), the method returns false and the function returns empty image (with %cv::Mat, test it with Mat::empty()).
.   
.       @note In @ref videoio_c "C API", functions cvRetrieveFrame() and cv.RetrieveFrame() return image stored inside the video
.       capturing structure. It is not allowed to modify or release the image! You can copy the frame using
.       cvClon

In [ ]:
while True:
    ret, frame = vid.read()
    print(frame)
    if not ret:
        cv2.imshow('Dino', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            vid.release()
            cv2.destroyAllWindows()



In [16]:
vid.release()
cv2.destroyAllWindows()